In [ ]:
import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torchvision.models as models

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
import nltk
import re
from nltk.corpus import stopwords
from tensorflow import keras
import gensim.models
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
import tensorflow as tf
from tensorflow.keras import initializers, preprocessing
import tensorflow_hub as hub
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score, ConfusionMatrixDisplay, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
# !pip install bert-tensorflow
from bert.tokenization import FullTokenizer
# !pip install ekphrasis
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# !pip install emoji
import emoji
import re
import random

from pandas import DataFrame, read_csv
import gc          # Garbage collector
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import data
df = pd.read_table("/content/drive/MyDrive/ADL-proj/english_dataset.tsv", on_bad_lines='skip')

In [ ]:
df.head()

,id,tweet,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE


Filter

In [ ]:
def replace_digits_emojis(s):
    s = s.lower().strip()
    s = emoji.demojize(s)
    s = re.sub(r'\d+', '', s)
    s = re.sub(r'[^\w\s]', '', s)
    s = s.strip()
    return s

def remove_urls_mentions(text):
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    text = text.replace("RT", "").strip()
    return text

def replace_space(text):
    text = text.replace("\n", " ").strip()
    text = re.sub(r"\s+", ' ', text)
    text = text.strip()
    return text

def merge_outputs(processed_text):
    text = ""
    for l in processed_text:
        if "</" in l:
            l = l.replace("</", "<")

        if l in ['<percent>', '<url>', '<', '<number>', '</allcaps>',
                     '<money>', '<phone>', '<allcaps>', '<repeated>',  '<hashtag>',
                      '<date>', '<time>', '<censored>', '</hashtag>', '<email>']:
            continue
        elif l in ['<emphasis>', '<user>', '<surprise>',  '<laugh>', '<sad>', '<annoyed>', '<happy>']:
            if l == '<user>':
                continue
            else:
                text += " " + l
        else:
            text += " " + replace_digits_emojis(l)
    normalized = replace_space(text)
    return normalized

def normalize_text(input_text:str, text_preprocessor):
    processed_text = text_preprocessor.pre_process_doc(input_text)
    normalized_text = merge_outputs(processed_text)

    return normalized_text

In [ ]:
def sample_validation_set(X, y, ids):
    validation_sample_size = int((float(len(ids)) * 0.1)/2)

    X_train = {}
    y_train = {}
    y_train_ids = []
    X_valid = {}
    y_valid = {}
    y_valid_ids = []

    sampled_indexes = {0:[], 1:[]}
    index_counter = 0
    for label in y['output_label']:
        if len(sampled_indexes[label]) < validation_sample_size:
            sampled_indexes[label].append(index_counter)
        index_counter+=1


    for k in X:
        data = X[k]

        training_data = []
        validation_data = []
        index_counter = 0
        for d in data:
            label = y['output_label'][index_counter]

            # add to validation split
            if index_counter in sampled_indexes[label]:
                validation_data.append(d)
            else:
                training_data.append(d)

            index_counter +=1

        X_train[k] = np.array(training_data)
        X_valid[k] = np.array(validation_data)

    for k in y:
        data = y[k]

        training_data = []
        validation_data = []
        index_counter = 0
        for d in data:
            label = y['output_label'][index_counter]

            # add to validation split
            if index_counter in sampled_indexes[label]:
                validation_data.append(d)
            else:
                training_data.append(d)

            index_counter +=1

        y_train[k] = np.array(training_data)
        y_valid[k] = np.array(validation_data)

    index_counter = 0
    for id in ids:
        label = y['output_label'][index_counter]

        # add to validation split
        if index_counter in sampled_indexes[label]:
            y_valid_ids.append(id)
        else:
            y_train_ids.append(id)

        index_counter += 1

    return X_train, y_train, y_train_ids, X_valid, y_valid, y_valid_ids

Balance out Datasets : Oversampling

In [ ]:
def apply_oversampling(ids, labels, text_docs):

    count = {'HOF':0, 'NOT':0}
    label_to_ids = {'HOF':[], 'NOT':[]}

    c = 0
    for l in labels:
        count[l] +=1

        id = ids[c]
        label_to_ids[l].append(id)
        c+=1

    oversampled_ids, oversampled_labels, oversampled_text_docs = [], [], []

    if count['HOF'] > count['NOT']:
        max_label = 'HOF'
        min_label = 'NOT'
    else:
        max_label = 'NOT'
        min_label = 'HOF'


    label_diff = count[max_label] - count[min_label]

    random_ids = random.sample(label_to_ids[min_label], label_diff)

    for r in random_ids:
        id_index = ids.index(r)

        oversampled_ids.append(ids[id_index])
        oversampled_labels.append(labels[id_index])
        oversampled_text_docs.append(text_docs[id_index])

    # add the existing data
    oversampled_ids.extend(ids)
    oversampled_text_docs.extend(text_docs)
    oversampled_labels.extend(labels)

    return oversampled_ids, oversampled_labels, oversampled_text_docs

Tokenize, Padding

In [ ]:
def tokenize(text):
    tags = ['<emphasis>', '<user>', '<surprise>', '<percent>', '<url>', '<', '<number>', '</allcaps>', '<money>',
                 '<phone>', '<allcaps>', '<repeated>', '<laugh>', '<hashtag>', '<elongated>', '<sad>', '<annoyed>',
                 '<date>', '<time>', '<censored>', '<happy>', '</hashtag>', '<email>']
    tokens = text.split(' ')
    filtered_tokens = []

    for t in tokens:
        if t not in tags:
            filtered_tokens.append(t)
    return filtered_tokens

def pad_text(max_seq_len, token_ids):
    token_ids = token_ids[:min(len(token_ids), max_seq_len - 2)]
    token_ids = token_ids + [0] * (max_seq_len - len(token_ids))
    return np.array(token_ids)

Embedding

In [ ]:
def embed_text_with_hate_words(config, data: list, hate_words: list):
    x = list()
    for text in data:

        # tokenize
        tokens = text.split(' ')
        multihot_encoding_array = np.zeros(len(hate_words), dtype=int)

        for t in tokens:
            if t in hate_words:
                index = hate_words.index(t)
                multihot_encoding_array[index] = 1

        x.append(multihot_encoding_array)
    return np.array(x)

def embed_text_with_bert(config: dict, data: list, bert_tokenizer: FullTokenizer):
    x = list()

    for text in data:

        # tokenize
        tokens = bert_tokenizer.tokenize(text)
        tokens = ["[CLS]"] + tokens + ["[SEP]"]
        # convert tokens into IDs by embedding the text with BERT
        token_ids = bert_tokenizer.convert_tokens_to_ids(tokens)
        # pad zeros to the token ids, if necessary
        max_seq_len = config['tweet_text_seq_len']
        token_ids = pad_text(max_seq_len, token_ids)
        x.append(token_ids)
    return np.array(x)

def embed_text_with_characters(config: dict, data: list):
    char_tokenizer = Tokenizer(lower=True, char_level=True, oov_token="UNKNOWN")

    alphabet = " abcdefghijklmnopqrstuvwxyz"
    char_dict = {"PADDING": 0, "UNKNOWN": 1}
    for i, char in enumerate(alphabet):
        char_dict[char] = len(char_dict)

    char_tokenizer.word_index = char_dict

    x = char_tokenizer.texts_to_sequences(data)

    x_padded = pad_sequences(x, padding='post', maxlen=config['tweet_text_char_len'])

    return x_padded


In [ ]:
def normalize_text_docs(text_docs:list, text_preprocessor):
    normalized_text_docs = []
    for text in text_docs:
        normalized_text = normalize_text(text, text_preprocessor)
        normalized_text_docs.append(normalized_text)
    return normalized_text_docs

def encode_labels(data: list):
    y = list()
    label_to_index = {"HOF": 1, "NOT": 0}

    for label in data:
        y.append(label_to_index[label])
    return np.array(y)

In [ ]:
    # load the Ekphrasis preprocessor
    text_preprocessor = TextPreProcessor(
        # terms that will be normalized
        normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
                   'time', 'url', 'date', 'number'],
        # terms that will be annotated
        annotate={"hashtag", "allcaps", "elongated", "repeated",
                  'emphasis', 'censored'},
        fix_html=True,  # fix HTML tokens

        # corpus from which the word statistics are going to be used
        # for word segmentation
        segmenter="twitter",

        # corpus from which the word statistics are going to be used
        # for spell correction
        corrector="twitter",

        unpack_hashtags=True,  # perform word segmentation on hashtags
        unpack_contractions=True,  # Unpack contractions (can't -> can not)
        spell_correct_elong=False,  # spell correction for elongated words

        # select a tokenizer. You can use SocialTokenizer, or pass your own
        # the tokenizer, should take as input a string and return a list of tokens
        tokenizer=SocialTokenizer(lowercase=True).tokenize,

        # list of dictionaries, for replacing tokens extracted from the text,
        # with other expressions. You can pass more than one dictionaries.
        dicts=[emoticons]
    )

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


In [ ]:
for index, row in df.iterrows():
  row['tweet'] = replace_digits_emojis(row['tweet'])
  row['tweet'] = remove_urls_mentions(row['tweet'])
  row['tweet'] = replace_space(row['tweet'])
  row['tweet'] = merge_outputs(row['tweet'])
  row['tweet'] = normalize_text(row['tweet'], text_preprocessor)
df.head()

,id,tweet,task_1,task_2,task_3
0,hasoc_en_1,d h o n i k e e p s t h e g l o v e w a t c h ...,NOT,NONE,NONE
1,hasoc_en_2,p o l i t i c o n o w e s h o u l d r e m e m ...,HOF,HATE,TIN
2,hasoc_en_3,c r i c k e t w o r l d c u p g u e s s w h o ...,NOT,NONE,NONE
3,hasoc_en_4,c o r b y n i s t o o p o l i t i c a l l y i ...,NOT,NONE,NONE
4,hasoc_en_5,a l l t h e b e s t t o t e a m i n d i a f o ...,NOT,NONE,NONE


Preprocessing


In [ ]:
# Spliting up the data into test and training set
from sklearn.model_selection import train_test_split
train_set, test_set = np.split(df.sample(frac=1), [int(.9*len(df))])

In [ ]:
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

tokenizer=''
def preprocess_data_for_rnn(data):
    global tokenizer

    #1. Tokenize the data
    tokenizer = Tokenizer(num_words=10000)
    tokenizer.fit_on_texts(data['inputs'])
    #Tensorflow’s tokenizer assigns a unique token to each distinct word. 

    #2. Transform/Convert text to sequence
    train = tokenizer.texts_to_sequences(data['inputs'])

    #3. Apply padding: padding is done to get all the data to the same length so as to send it to an RNN layer. 
    x_train = pad_sequences(train)

    #4. encoding the outputs: target variables are also encoded to decimal values. 
    le = LabelEncoder()
    y_train = le.fit_transform(data['tags'])
    
    vocabulary, input_shape, output_length=get_model_input_values(x_train, tokenizer, le)
    
    return x_train, y_train, vocabulary, input_shape, output_length

def get_model_input_values(x_train, tokenizer, le):
    #input length
    input_shape = x_train.shape[0]
    print("Input shape: ", input_shape)
    #define vocabulary
    vocabulary = len(tokenizer.word_index)
    print("number of unique words in vocabulary: ",vocabulary)
    #output length
    output_length = le.classes_.shape[0]
    print("output length: ",output_length)
    
    return vocabulary, input_shape, output_length

In [ ]:
# Define Model: creating the model
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model

def get_rnn_model(input_shape, vocabulary):

    i = Input(shape=(input_shape,))
    x = Embedding(vocabulary+1,333)(i)
    x = LSTM(333,return_sequences=True)(x)
    x = Flatten()(x)
    x = Dense(output_length,activation="softmax")(x)
    model  = Model(i,x)
    #compiling the model
    model.compile( loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
   
    return model

In [ ]:
train_set = train_set[['tweet', 'task_1']].astype(str).rename(columns={'tweet': 'inputs', 'task_1': 'tags'})
test_set = test_set[['tweet', 'task_1']].astype(str).rename(columns={'tweet': 'inputs', 'task_1': 'tags'})
train_set.head()
print("Total Samples to train:", len(train_set))
print("Total Samples to test:", len(test_set))

Total Samples to train: 5266
Total Samples to test: 586


In [ ]:
train_set.head(5)

,inputs,tags
5342,Your orange bigoted ASSHOLE boss throws babies...,HOF
3813,.@drharshvardhan Sur .. make central govt ru...,NOT
5635,@realDonaldTrump @MarshaBlackburn #TrumpMustRe...,HOF
1817,"This movement is HEROIC, it is revolution agai...",NOT
4258,@iamcardib acts like such a badass then goes a...,HOF


In [ ]:
%time x_train, y_train, vocabulary, input_shape, output_length = preprocess_data_for_rnn(train_set)
model = get_rnn_model(input_shape, vocabulary)

Input shape:  5266
number of unique words in vocabulary:  20259
output length:  2
CPU times: user 352 ms, sys: 2.95 ms, total: 355 ms
Wall time: 357 ms


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5266)]            0         
                                                                 
 embedding (Embedding)       (None, 5266, 333)         6746580   
                                                                 
 lstm (LSTM)                 (None, 5266, 333)         888444    
                                                                 
 flatten (Flatten)           (None, 1753578)           0         
                                                                 
 dense (Dense)               (None, 2)                 3507158   
                                                                 
Total params: 11,142,182
Trainable params: 11,142,182
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Train and validate the model
from tensorflow.keras.callbacks import EarlyStopping
%time history = model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.3, callbacks=[EarlyStopping(monitor='val_loss', patience=2, min_delta=0.0001)])

Epoch 1/5


ValueError: ignored

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show();

In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show();

In [ ]:
for sentences in test_set.inputs.values:
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in sentences if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  print(prediction_input)
  #texts_p.append(prediction_input)

  #tokenizing and padding
  print(prediction_input)
  c = model.predict(prediction_input)
  break